In [1]:
import pandas as pd
import numpy as np
import design_class_dictionary

In [ ]:
master = pd.read_csv('data/master_503107.csv')

# All classes to all classes

In [ ]:
d2d = pd.read_csv('data/final_main_d2d.csv')
d2d.head()

In [ ]:
d2u = pd.read_csv('data/final_main_d2u.csv')
d2u.head()

# First listed class of focal patent to all listed in cited patents

In [2]:
pat_class = pd.read_csv('data/uspc_current.tsv', delimiter = '\t', usecols=['patent_id','mainclass_id','sequence'],\
                       dtype={'patent_id':str})
pat_class.head()

,patent_id,mainclass_id,sequence
0,3930271,2,0
1,3930271,2,1
2,3930271,2,2
3,3930271,473,3
4,3930272,5,0


In [3]:
#select first listed sequence 
first_listed = pat_class.loc[(pat_class.sequence == 0)]

Realizing that the cited patent number is necessary to have. Pulling other dataframe with this information

In [ ]:
final = pd.read_csv('data/final_no_drops.csv')
cols = ['cite_subclass','focal_subclass']
final.drop(cols, axis=1, inplace=True)
final.head()

In [ ]:
final.patent_number.nunique()

In [ ]:
#get relevant patents
final = final.loc[final.patent_number.isin(master.patent_number.tolist())]

Let's test this compared to the d2d and d2u dataframes to see if they get the same end result

In [ ]:
# drop duplicate rows. These are most likely unique on a subclass level, but not londer on a mainclass level
final = final.drop_duplicates()

In [ ]:
# attribute first listed class to focal patent and cited patents
final_f2a = pd.merge(final, first_listed, left_on='patent_number', right_on='patent_id', how='left')

In [ ]:
final_f2a.head()

In [ ]:
#split into design and untilty cited patents, and remove irregular citations
final_f2a_d2d = final_f2a.loc[final_f2a.cited_patent_number.str.contains('D')]
final_f2a_d2d = final_f2a_d2d.loc[final_f2a_d2d.cite_mainclass.str.contains('D')]

final_f2a_d2u = final_f2a.loc[~final_f2a.cited_patent_number.str.contains('D')]
final_f2a_d2u = final_f2a_d2u.loc[~final_f2a_d2u.cite_mainclass.str.contains('D')]

In [ ]:
cols = ['cited_patent_number','focal_mainclass','patent_id','sequence']
final_f2a_d2d = final_f2a_d2d.drop(cols, axis=1).drop_duplicates()
final_f2a_d2u = final_f2a_d2u.drop(cols, axis=1).drop_duplicates()

In [ ]:
print(final_f2a_d2d.mainclass_id.unique(),'\n\n', final_f2a_d2d.cite_mainclass.unique(),'\n\n',\
      final_f2a_d2u.mainclass_id.unique(), '\n\n', final_f2a_d2u.cite_mainclass.unique())

In [ ]:
final_f2a_d2d[['mainclass_id','cite_mainclass']] = final_f2a_d2d[['mainclass_id','cite_mainclass']]\
.replace(design_class_dictionary.class_dictionary)

final_f2a_d2d.dropna(subset=['mainclass_id'], inplace=True)
final_f2a_d2d = final_f2a_d2d.loc[final_f2a_d2d.mainclass_id.str.contains('D')].drop_duplicates()

#########################################################################################

final_f2a_d2u['mainclass_id'] = final_f2a_d2u['mainclass_id']\
.replace(design_class_dictionary.class_dictionary)

final_f2a_d2u.dropna(subset=['mainclass_id'], inplace=True)
final_f2a_d2u = final_f2a_d2u.loc[final_f2a_d2u.mainclass_id.str.contains('D')]

final_f2a_d2u = final_f2a_d2u.loc[~(final_f2a_d2u.cite_mainclass.str.contains('PLT'))\
                  & (~final_f2a_d2u.cite_mainclass.str.contains('No'))].drop_duplicates()

# First listed focal class to first listed cited class

In [ ]:
final_f2f = pd.merge(final_f2a, first_listed, left_on='cited_patent_number', right_on='patent_id', how='left')

In [ ]:
final_f2f.head()

In [ ]:
#split between two measures and clean up
final_f2f_d2d = final_f2f.loc[final_f2f.cited_patent_number.str.contains('D')]
final_f2f_d2d = final_f2f_d2d.loc[final_f2f_d2d.mainclass_id_y.str.contains('D')]

final_f2f_d2u = final_f2f.loc[~final_f2f.cited_patent_number.str.contains('D')]
# final_f2f_d2u = final_f2f_d2u.loc[~final_f2f_d2u.mainclass_id_y.str.contains('D')]


In [ ]:
final_f2f_d2d.head()

In [ ]:
cols=['cited_patent_number','cite_mainclass','focal_mainclass','patent_id_x','sequence_x','patent_id_y','sequence_y']

final_f2f_d2d = final_f2f_d2d.drop(cols, axis=1).drop_duplicates()
final_f2f_d2u = final_f2f_d2u.drop(cols, axis=1).drop_duplicates()

In [ ]:
final_f2f_d2d.head()

In [ ]:
print(final_f2f_d2d.mainclass_id_x.unique(),'\n\n', final_f2f_d2d.mainclass_id_y.unique(),'\n\n',\
      final_f2f_d2u.mainclass_id_x.unique(), '\n\n', final_f2f_d2u.mainclass_id_y.unique())

In [ ]:
final_f2f_d2d[['mainclass_id_x','mainclass_id_y']] = final_f2f_d2d[['mainclass_id_x','mainclass_id_y']]\
.replace(design_class_dictionary.class_dictionary)

final_f2f_d2d.dropna(subset=['mainclass_id_x'], inplace=True)
final_f2f_d2d = final_f2f_d2d.loc[final_f2f_d2d.mainclass_id_x.str.contains('D')].drop_duplicates()

##########################################################################################################

final_f2f_d2u['mainclass_id_x'] = final_f2f_d2u['mainclass_id_x'].replace(design_class_dictionary.class_dictionary)
final_f2f_d2u.dropna(inplace=True)
final_f2f_d2u = final_f2f_d2u.loc[final_f2f_d2u.mainclass_id_x.str.contains('D')]

final_f2f_d2u = final_f2f_d2u.loc[(~final_f2f_d2u.mainclass_id_y.str.contains('PLT'))\
                  & (~final_f2f_d2u.mainclass_id_y.str.contains('No'))\
                 & (~final_f2f_d2u.mainclass_id_y.str.contains('D'))].drop_duplicates()

In [ ]:
final_f2a_d2d.rename(index=str, columns={'mainclass_id':'focal_mainclass'}, inplace=True)
final_f2a_d2u.rename(index=str, columns={'mainclass_id':'focal_mainclass'}, inplace=True)

In [ ]:
final_f2f_d2d.rename(index=str, columns={'mainclass_id_x':'focal_mainclass', 'mainclass_id_y':'cite_mainclass'},\
                     inplace=True)
final_f2f_d2u.rename(index=str, columns={'mainclass_id_x':'focal_mainclass', 'mainclass_id_y':'cite_mainclass'},\
                     inplace=True)

# Now that all the datasets are set up, let's throw them into the outside functions

In [ ]:
d2d.reset_index(drop=True).to_csv('data/comparison/all_to_all_d2d.csv', index_label=False)
d2u.reset_index(drop=True).to_csv('data/comparison/all_to_all_d2u.csv', index_label=False)

final_f2a_d2d.reset_index(drop=True).to_csv('data/comparison/first_to_all_d2d.csv', index_label=False)
final_f2a_d2u.reset_index(drop=True).to_csv('data/comparison/first_to_all_d2u.csv', index_label=False)

final_f2f_d2d.reset_index(drop=True).to_csv('data/comparison/first_to_first_d2d.csv', index_label=False)
final_f2f_d2u.reset_index(drop=True).to_csv('data/comparison/first_to_first_d2u.csv', index_label=False)

# Ran the data, now calculate correlation matix of the measures at the industry level

Design to design

In [4]:
all_to_all = pd.read_csv('data/comparison/all_to_all_d2d_output.csv')
first_to_all = pd.read_csv('data/comparison/first_to_all_d2d_output.csv')
first_to_first = pd.read_csv('data/comparison/first_to_first_d2d_output.csv')

In [5]:
first_to_first_count = first_to_first.loc[first_to_first['first_seen'] == 1]\
.groupby('focal_mainclass')['patent_number'].count()\
.reset_index().rename(index=str, columns={'patent_number':'first to first novelty count'})
first_to_first_count = first_to_first_count.append(first_to_first_count.sum(numeric_only=True), ignore_index=True)

first_to_first_mean = first_to_first.groupby('focal_mainclass')['first_seen'].mean().reset_index()\
.rename(index=str, columns={'first_seen':'first to first mean novelty'})
first_to_first_mean = first_to_first_mean.append(first_to_first_mean.mean(numeric_only=True), ignore_index=True)\
.round(3)

first_to_first_table = pd.merge(first_to_first_count, first_to_first_mean)


In [6]:
first_to_all_count = first_to_all.loc[first_to_all['first_seen'] == 1]\
.groupby('focal_mainclass')['patent_number'].count()\
.reset_index().rename(index=str, columns={'patent_number':'first to all novelty count'})
first_to_all_count = first_to_all_count.append(first_to_all_count.sum(numeric_only=True), ignore_index=True)

first_to_all_mean = first_to_all.groupby('focal_mainclass')['first_seen'].mean().reset_index()\
.rename(index=str, columns={'first_seen':'first to all mean novelty'})
first_to_all_mean = first_to_all_mean.append(first_to_all_mean.mean(numeric_only=True), ignore_index=True)\
.round(3)

first_to_all_table = pd.merge(first_to_all_count, first_to_all_mean)


In [7]:
all_to_all = pd.merge(all_to_all, first_listed, how='left', left_on='patent_number', right_on='patent_id')
all_to_all['mainclass_id'] = all_to_all['mainclass_id'].replace(design_class_dictionary.class_dictionary)
all_to_all.dropna(subset=['mainclass_id'], inplace=True)
all_to_all = all_to_all.loc[all_to_all['mainclass_id'].str.contains('D')]

all_to_all_count = all_to_all.loc[all_to_all.first_seen == 1]\
.groupby('mainclass_id')['patent_number'].count().reset_index()\
.rename(index=str, columns={'patent_number':'all to all novelty count'})
all_to_all_count = all_to_all_count.append(all_to_all_count.sum(numeric_only=True), ignore_index=True)

all_to_all_mean = all_to_all.groupby('mainclass_id')['first_seen'].mean().reset_index()\
.rename(index=str, columns = {'first_seen':'all to all mean novelty'})
all_to_all_mean = all_to_all_mean.append(all_to_all_mean.mean(numeric_only=True), ignore_index= True).round(3)

all_to_all_table = pd.merge(all_to_all_count, all_to_all_mean)

In [8]:
all_to_all_table.head()

,mainclass_id,all to all novelty count,all to all mean novelty
0,D01,15.0,0.002
1,D02,43.0,0.002
2,D03,64.0,0.002
3,D04,33.0,0.004
4,D05,20.0,0.004


In [9]:
first_to_all_table.head()

,focal_mainclass,first to all novelty count,first to all mean novelty
0,D01,17.0,0.003
1,D02,48.0,0.002
2,D03,37.0,0.001
3,D04,34.0,0.005
4,D05,16.0,0.004


In [10]:
first_to_first_table.head()

,focal_mainclass,first to first novelty count,first to first mean novelty
0,D01,29.0,0.006
1,D02,48.0,0.002
2,D03,40.0,0.002
3,D04,32.0,0.006
4,D05,21.0,0.006


In [11]:
total_table = pd.merge(all_to_all_table, first_to_all_table, left_on='mainclass_id', right_on='focal_mainclass')

total_table = pd.merge(total_table, first_to_first_table)

total_table.head()

,mainclass_id,all to all novelty count,all to all mean novelty,focal_mainclass,first to all novelty count,first to all mean novelty,first to first novelty count,first to first mean novelty
0,D01,15.0,0.002,D01,17.0,0.003,29.0,0.006
1,D02,43.0,0.002,D02,48.0,0.002,48.0,0.002
2,D03,64.0,0.002,D03,37.0,0.001,40.0,0.002
3,D04,33.0,0.004,D04,34.0,0.005,32.0,0.006
4,D05,20.0,0.004,D05,16.0,0.004,21.0,0.006


In [12]:
total_table.drop(labels=['mainclass_id'], axis=1, inplace=True)

total_table['class name'] = total_table['focal_mainclass'].map(design_class_dictionary.class_labels)

In [13]:
#reorder columns for readability
cols=['focal_mainclass','class name','all to all novelty count','all to all mean novelty',\
      'first to all novelty count','first to all mean novelty','first to first novelty count',\
      'first to first mean novelty']
total_table = total_table[cols]

In [14]:
total_table

,focal_mainclass,class name,all to all novelty count,all to all mean novelty,first to all novelty count,first to all mean novelty,first to first novelty count,first to first mean novelty
0,D01,Edible Products,15.0,0.002,17.0,0.003,29.0,0.006
1,D02,Apparel and Haberdashery,43.0,0.002,48.0,0.002,48.0,0.002
2,D03,"Travel Goods, Personal Belongings, and Storage...",64.0,0.002,37.0,0.001,40.0,0.002
3,D04,Brushware,33.0,0.004,34.0,0.005,32.0,0.006
4,D05,Textile or Paper Yard Goods; Sheet Material,20.0,0.004,16.0,0.004,21.0,0.006
5,D06,Furnishings,108.0,0.001,67.0,0.001,44.0,0.001
6,D07,Equipment for Preparing or Serving Food or Dri...,69.0,0.001,48.0,0.001,22.0,0.000
7,D08,Tools and Hardware,46.0,0.001,36.0,0.001,27.0,0.001
8,D09,Packages and Containers for Goods,65.0,0.001,54.0,0.001,52.0,0.001
9,D10,"Measuring, Testing or Signaling Instruments",45.0,0.001,32.0,0.001,31.0,0.001


In [15]:
#save it
total_table.to_csv('data/d2d_total_table.csv', index_label=False)
total_table.corr().round(3).to_csv('data/d2d_total_table_correlations.csv')

Design to utility

In [16]:
all_to_all = pd.read_csv('data/comparison/all_to_all_d2u_output.csv')
first_to_all = pd.read_csv('data/comparison/first_to_all_d2u_output.csv')
first_to_first = pd.read_csv('data/comparison/first_to_first_d2u_output.csv')

In [17]:
first_to_first_count = first_to_first.loc[first_to_first['first_seen'] == 1]\
.groupby('focal_mainclass')['patent_number'].count()\
.reset_index().rename(index=str, columns={'patent_number':'first to first novelty count'})
first_to_first_count = first_to_first_count.append(first_to_first_count.sum(numeric_only=True), ignore_index=True)

first_to_first_mean = first_to_first.groupby('focal_mainclass')['first_seen'].mean().reset_index()\
.rename(index=str, columns={'first_seen':'first to first mean novelty'})
first_to_first_mean = first_to_first_mean.append(first_to_first_mean.mean(numeric_only=True), ignore_index=True)\
.round(3)

first_to_first_table = pd.merge(first_to_first_count, first_to_first_mean)

In [18]:
first_to_all_count = first_to_all.loc[first_to_all['first_seen'] == 1]\
.groupby('focal_mainclass')['patent_number'].count()\
.reset_index().rename(index=str, columns={'patent_number':'first to all novelty count'})
first_to_all_count = first_to_all_count.append(first_to_all_count.sum(numeric_only=True), ignore_index=True)

first_to_all_mean = first_to_all.groupby('focal_mainclass')['first_seen'].mean().reset_index()\
.rename(index=str, columns={'first_seen':'first to all mean novelty'})
first_to_all_mean = first_to_all_mean.append(first_to_all_mean.mean(numeric_only=True), ignore_index=True)\
.round(3)

first_to_all_table = pd.merge(first_to_all_count, first_to_all_mean)


In [19]:
all_to_all = pd.merge(all_to_all, first_listed, how='left', left_on='patent_number', right_on='patent_id')
all_to_all['mainclass_id'] = all_to_all['mainclass_id'].replace(design_class_dictionary.class_dictionary)
all_to_all.dropna(subset=['mainclass_id'], inplace=True)
all_to_all = all_to_all.loc[all_to_all['mainclass_id'].str.contains('D')]

all_to_all_count = all_to_all.loc[all_to_all.first_seen == 1]\
.groupby('mainclass_id')['patent_number'].count().reset_index()\
.rename(index=str, columns={'patent_number':'all to all novelty count'})
all_to_all_count = all_to_all_count.append(all_to_all_count.sum(numeric_only=True), ignore_index=True)

all_to_all_mean = all_to_all.groupby('mainclass_id')['first_seen'].mean().reset_index()\
.rename(index=str, columns = {'first_seen':'all to all mean novelty'})
all_to_all_mean = all_to_all_mean.append(all_to_all_mean.mean(numeric_only=True), ignore_index= True).round(3)

all_to_all_table = pd.merge(all_to_all_count, all_to_all_mean)

In [20]:
total_table = pd.merge(all_to_all_table, first_to_all_table, left_on='mainclass_id', right_on='focal_mainclass')

total_table = pd.merge(total_table, first_to_first_table)

total_table.head()

,mainclass_id,all to all novelty count,all to all mean novelty,focal_mainclass,first to all novelty count,first to all mean novelty,first to first novelty count,first to first mean novelty
0,D01,252.0,0.049,D01,309.0,0.073,194.0,0.084
1,D02,398.0,0.013,D02,450.0,0.015,349.0,0.018
2,D03,500.0,0.010,D03,491.0,0.011,378.0,0.017
3,D04,273.0,0.021,D04,334.0,0.030,224.0,0.038
4,D05,362.0,0.046,D05,398.0,0.055,281.0,0.084


In [21]:
total_table.drop(labels=['mainclass_id'], axis=1, inplace=True)

total_table['class name'] = total_table['focal_mainclass'].map(design_class_dictionary.class_labels)

cols=['focal_mainclass','class name','all to all novelty count','all to all mean novelty',\
      'first to all novelty count','first to all mean novelty','first to first novelty count',\
      'first to first mean novelty']
total_table = total_table[cols]

In [22]:
total_table

,focal_mainclass,class name,all to all novelty count,all to all mean novelty,first to all novelty count,first to all mean novelty,first to first novelty count,first to first mean novelty
0,D01,Edible Products,252.0,0.049,309.0,0.073,194.0,0.084
1,D02,Apparel and Haberdashery,398.0,0.013,450.0,0.015,349.0,0.018
2,D03,"Travel Goods, Personal Belongings, and Storage...",500.0,0.010,491.0,0.011,378.0,0.017
3,D04,Brushware,273.0,0.021,334.0,0.030,224.0,0.038
4,D05,Textile or Paper Yard Goods; Sheet Material,362.0,0.046,398.0,0.055,281.0,0.084
5,D06,Furnishings,740.0,0.009,674.0,0.009,504.0,0.012
6,D07,Equipment for Preparing or Serving Food or Dri...,529.0,0.007,533.0,0.008,470.0,0.013
7,D08,Tools and Hardware,921.0,0.009,695.0,0.008,537.0,0.010
8,D09,Packages and Containers for Goods,481.0,0.006,537.0,0.007,447.0,0.011
9,D10,"Measuring, Testing or Signaling Instruments",686.0,0.016,554.0,0.015,442.0,0.023


In [23]:
#save it
total_table.to_csv('data/d2u_total_table.csv', index_label=False)
total_table.corr().round(3).to_csv('data/d2u_total_table_correlations.csv')